# Смысла использовать нейронную сеть не оказалось из-за малого количества обучающих данных, сеть сразу же отказалась сходиться. Саму сеть выкладываем ниже без препроцессинга.

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Определяем входную послеловательность
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# Оставляем только стейты внутри лстмки
encoder_states = [state_h, state_c]

# Устанавливаем декодер на основе стейтов инкодера
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# Декодер возвращает фулстейт
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Определяем модель
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Тренируем
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

In [ ]:
# Настраиваем инференс
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
# Получаем протокол
def decode_sequence(input_seq):
    # Кодируем вектор
    states_value = encoder_model.predict(input_seq)

    # Генерим пустую цель
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Цикл для семплинга
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Сэмплим токен
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Выходим по длине или стопсимволу
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Увеличиваем выходную последовательность
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Обновляем стейты
        states_value = [h, c]

    return decoded_sentence